# Beat Populations

Link census tract populations, total and by race, into police beats. Attributes population from tracts to beats by the areas of the overlaps. The basic procedure is to find the overlaps between beats and Census tracts, then addign a portion of the population of the tract to the beat, based on the raio of the size of overlap to the size of the tract. 

In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
import utm

%matplotlib inline
sns.set_context('notebook')


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# San Diego Police Regions and Demographics
`sandiego.gov-police_regions-2.1.1` Last Update: 2021-02-25T18:10:53

_Boundary shapes for San Diego neighborhoods, beats and divisions, with ACS 2019 estimates for populations, by race._


This package links shapefiles for San Diego police beats to Census tracts and
merges in ACS estimates for population, by race, from the 2016 5 year ACS. When
a police beat boundry crosses a tract, the tract population is allocated to
beats by the proportion of the overlap by area. See the [Jupyter
notebook](https://github.com/metatab-packages/metatab-packages/blob/master/sandi
ego.gov/sandiego.gov-police_regions/notebooks/BeatPopulations.ipynb) that
performs the procedure for details.

For the race/ethicty statistics, Hispanic ('hisp') refers to Hispanics of any
race, while all other races refer to non-Hispanics of that race.
## Documentation Links

* [Police Divisions Repository Page](https://data.sandiego.gov/datasets/pd-divisions/) Data repository page that links to original files.

 
## Notes

* Version 2.1.1 Updates to 2019 Census, better EPSG ( UTM 11N ) for area calc
## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [pd_beats](notebooks/beats.ipynb#pd_beats)**. Police beats
* ** [pd_divisions](notebooks/beats.ipynb#pd_divisions)**. Police Divisions
* ** [pd_neighborhoods](notebooks/beats.ipynb#pd_neighborhoods)**. Police Neighborhoods
* ** [beat_demographics](notebooks/BeatPopulations.ipynb#beat_demographics)**. Counts of people in the beat, by race.

## References
<ul><li> <strong>tracts</string>, <em>censusgeo://2019/5/CA/tract</em>. Census tracts from 2016 5 year ACS, for San Diego county</li><li> <strong>race</string>, <em>census://2019/5/CA/tract/B03002</em>. Race, by tract, in San Diego county</li><li> <strong>pd_beats_source</string>, <em>shape+http://seshat.datasd.org/sde/pd/pd_beats_datasd.zip</em>. Police beats</li><li> <strong>pd_divisions_source</string>, <em>shape+http://seshat.datasd.org/sde/pd/pd_divisions_datasd.zip</em>. Police Divisions</li><li> <strong>pd_neighborhoods_source</string>, <em>shape+http://seshat.datasd.org/sde/pd/pd_divisions_datasd.zip</em>. Police Neighborhoods</li><ul>

In [6]:
# Convert to EPSG:26911, ( A randomly selected UTM Zone 11N CRS) so area calculations 
# will be in square meters, rather than square degrees
beats = pkg.resource('pd_beats').geoframe()
beats.head()

,objectid,div_num,div_name,geometry
0,1,1,NORTHERN,"POLYGON ((-117.24145 32.91383, -117.23726 32.9..."
1,2,2,NORTHEASTERN,"POLYGON ((-117.00941 33.11314, -117.00942 33.1..."
2,3,3,EASTERN,"POLYGON ((-117.03127 32.83776, -117.03666 32.8..."
3,4,5,CENTRAL,"POLYGON ((-117.14225 32.74120, -117.14225 32.7..."
4,5,6,WESTERN,"POLYGON ((-117.16408 32.80326, -117.15802 32.8..."


In [4]:
# Figure out the UTM zone, and from that the CRS EPSG for the UTM zone
xmin, ymin, xmax, ymax = beats.to_crs(4326).envelope.total_bounds
band = utm.from_latlon((ymax-ymin)/2+ymin, (xmax-xmin)/2+xmin )[2]
epsg = int('326'+str(band))

assert(epsg == 32611) # It's always San Diego, so we already know what it is ... 


In [5]:
# Convert to a UTM EPSG, so that we can do calculations in meters. 
beats = beats.to_crs(epsg)

# There are  beats that are way off in east county. Get rid of them.
#rightmost_centroid = beats.centroid.x.sort_values(ascending=False).iloc[:6].max()
#beats = beats[beats.centroid.x <rightmost_centroid]

# It looks like the dataset has multiple rows per beat, one feature per row. We need
# it to have one row per beat, with multiple features combined together. 
beats = beats.dissolve(by='beat').reset_index()

#  Add the area
beats['beat_area'] = beats.area / 1_000_000

beats.plot()

KeyError: 'beat'

In [ ]:
tracts = pkg.reference('tracts').geoframe().to_crs(epsg)

#  Add the area
tracts['tract_area'] = tracts.area / 1_000_000


tracts.plot()

In [ ]:
# White, black, asian, etc are all non hispanic. 
col_map = {
    'b03002_001':'total',
    'b03002_003':'white',
    'b03002_004':'black',
    'b03002_005':'aian',
    'b03002_006':'asian',
    'b03002_007':'nhopi', 
    'b03002_012':'hisp'
    
}

for k,v in list(col_map.items()):
    col_map[k+'_m90'] = col_map[k]+'_m90'
    
t = pkg.reference('race').dataframe()
    
race_tracts = t[t.county=='073'].rename(columns=col_map).reset_index().rename(columns={'GEOID':'geoid'})

In [ ]:
race_tracts = race_tracts[['geoid', 'total', 'white', 'black', 'aian', 'asian', 'nhopi', 'hisp']]
race_tracts.titles.head().T

In [ ]:
t = gpd.sjoin(beats, tracts)

ax = t.plot()
beats.centroid.plot(ax=ax, color='red')

t = t[['geoid', 'beat']].drop_duplicates()\
    .merge(tracts[['geoid','geometry', 'tract_area']],on='geoid')\
    .merge(beats[['beat','geometry', 'beat_area']],on='beat')


In [ ]:
# merge by finding intersections. Each record in intr will be an area that is entirely
# in both one tract and one beat. 
intr = gpd.overlay(beats, tracts, how='intersection')[['beat','geoid','geometry']]
print(len(beats), len(tracts), len(intr))
intr['intr_area'] = (intr.geometry.area/1_000_000.0).astype(float)

# Get rid of really small intersections
intr = intr[intr.intr_area >= .01] 

# Merge back with beats and tracts to get beat and tract areas. 
merged = intr[['beat','geoid', 'intr_area']]\
    .merge(tracts[['geoid', 'tract_area']],on='geoid')\
    .merge(beats[['beat', 'beat_area']],on='beat')\
    .merge(race_tracts, on='geoid')

merged = merged.drop_duplicates(subset=['beat','geoid'])

merged['tract_overlap_proportion'] = merged.intr_area/merged.tract_area
merged['beat_overlap_proportion'] = merged.intr_area/merged.beat_area

# The intersection areas must be smaller than both of the areas being intersected
assert(not any(merged.intr_area > merged.beat_area))
assert(not any(merged.intr_area > merged.tract_area))

# Check that all of the areas of the beats are accounted for
assert(all(merged.groupby('beat').beat_overlap_proportion.sum().round(1) == 1))

merged['total'] = merged.total * merged.tract_overlap_proportion
merged['white'] = merged.white * merged.tract_overlap_proportion
merged['asian'] = merged.asian * merged.tract_overlap_proportion
merged['black'] = merged.black * merged.tract_overlap_proportion
merged['aian']  = merged.aian * merged.tract_overlap_proportion
merged['hisp']  = merged.hisp * merged.tract_overlap_proportion
merged['nhopi']  = merged.nhopi * merged.tract_overlap_proportion

merged.head().T


In [ ]:
beat_demographics = merged.groupby('beat').sum()[['total', 'white', 'black', 'aian', 'asian', 'nhopi', 'hisp']].round()